In [ ]:
run_path = "../data/runs/model268-epoch5.debiased/"
training_data = "../data/processed/training.nc"
training_data_mean = "../data/processed/training.mean.nc"
caseid = "control"
ngpath = "../data/raw/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc"

In [ ]:
from src.data.sam import SAMRun
from uwnet.thermo import vorcitity
from os.path import join
import holoviews as hv
hv.extension('bokeh')
plt.style.use('colorblind')

%opts Image[width=600, height=400, colorbar=True](cmap='viridis')

def plot_2d(da, dynamic=False):
    time = da.time
    da = da.assign_coords(time=time-time[0])
    max = da.time.max('time')
    da = da.load().interp(time=np.arange(max))
    return hv.Dataset(da).to.image(["x", "y"], dynamic=dynamic).redim.unit(time="day")


In [ ]:
# Load the data
run = SAMRun(run_path, case=caseid)
ng = xr.open_dataset(training_data).isel(step=0)
ng_2d = xr.open_dataset(ngpath).sortby('time')
mean_ng = xr.open_dataset(training_data_mean)
# mean_nn = run.data_3d.isel(time=slice(5,None)).mean(['x', 'time'])

# Hovmoller diagrams of zonal mean quantities

In [ ]:
run.data_2d.W500.mean('x').plot(y='y')

The vertical motion decreases for the first 130 days and then becomes much stronger.

In [ ]:
run.data_2d.PW.mean('x').plot(y='y')

In [ ]:
run.data_2d.NPNN.mean('x').plot(y='y')

In [ ]:
run.data_2d.U850.mean('x').plot(y='y')

The biggest problems seem to be because of the position of jet, which dramatically weakens and shifts towards the equator. The changes in the tropical circulation seem to be a knock-on effect of these circulation changes.

Here is the same plot for NG-AQua:

In [ ]:
ng_2d.U850.mean('x').plot(y='y')

The jet appears to be in statistical stead-state.

In [ ]:
d3 = run.data_3d

In [ ]:
def plot_evolution(da):
    da[1:24*20:24*5].mean('x').plot(col='time')

In [ ]:
plot_evolution(d3.FQTNN)

In [ ]:
plot_evolution(d3.FSLINN)

In [ ]:
plot_evolution(d3.U)

In [ ]:
plot_evolution(d3.W)

In [ ]:
d3.TABS[:24*20:24*5].mean('x').plot(col='time')

Why do the westerlies over the equator get so strong! Is it because of a lack of cumulus momentum transport or meridional transport?

# Evolution of U near the equator

In [ ]:
d3_evo = d3.isel(time=np.r_[:24*20:24])

d3_evo.U.isel(y=34).mean('x').plot(x='time', vmax=30)

In [ ]:
ng.U[:8*20:8,:,34].mean('x').plot(x='time', vmax=30)

The SAM+NN simulation develops extremely strong westerlies aloft and weaker near surface easterlies. This implies that the total momentum above the model equator increases. This cannot be caused by CMT, which re-distributes momentum within a column. This must be caused by *meridional transport*.

In [ ]:
uv = (d3_evo.U * d3_evo.V).mean('x').interp(z=15e3)

In [ ]:
uv.plot()